$
\newcommand{\red}[1]{{\color{red}{#1}}}
\newcommand{\green}[1]{{\color{green}{#1}}}
\newcommand{\blue}[1]{{\color{blue}{#1}}}
\newcommand{\grey}[1]{{\color{grey}{#1}}}
\newcommand{\orange}[1]{{\color{orange}{#1}}}
\newcommand{\msbr}[1]{\begin{bmatrix}#1\end{bmatrix}}
$

In [ ]:
%pip install -q ipympl
%matplotlib widget

In [ ]:
import math

import numpy as np

pi = np.pi
sqrt = np.sqrt
dot = lambda x, y: x.dot(y)
norm = lambda x: np.linalg.norm(x)

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from IPython.display import display, Latex, Markdown

import ipywidgets as widgets
from ipywidgets import FloatSlider, Label, Output, Layout

try:
    import google.colab
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    pass

def vec(*args):
    if len(args) == 1:
        return np.array(args[0], dtype=np.float64)
    else:
        return np.array(args, dtype=np.float64)

def vec_to_str(vec, row=True, fmt=".2g"):
    if row:
        return "\\begin{bmatrix} " + " \\\\ ".join((f"{{:{fmt}}}").format(val) for val in vec) + " \\end{bmatrix}"
    else:
        return "\\begin{bmatrix} " + " & ".join((f"{{:{fmt}}}").format(val) for val in vec) + " \\end{bmatrix}"

def vec_scale(vect, eps=0.1):
    return vect * (1.0 + eps/(np.linalg.norm(vect) + 0.01))
def vec_scale_2d(vect, off, eps=1.0, xlim=(-4.0, 4.0), ylim=(-4.0, 4.0)):
    return off + vect * (1.0 + eps/(np.linalg.norm(vect) + 0.01) * vec(xlim[1]-xlim[0], ylim[1]-ylim[0]))
def vec_scale_3d(vect, off, eps=0.1, xlim=(-2.0, 2.0), ylim=(-2.0, 2.0), zlim=(-2.0, 2.0)):
    return off + vect * (1.0 + eps/(np.linalg.norm(vect) + 0.01) * vec(xlim[1]-xlim[0], ylim[1]-ylim[0], zlim[1]-zlim[0]))
def vec_ratio(vect, rat=0.2):
    return rat / (np.linalg.norm(vect) + 0.01)
def vec_ratio_3d(vect, rat=0.2, xlim=(-2.0, 2.0), ylim=(-2.0, 2.0), zlim=(-2.0, 2.0)):
    return rat / (np.linalg.norm(vect) + 0.01) * norm(vec(xlim[1]-xlim[0], ylim[1]-ylim[0], zlim[1]-zlim[0]))

def draw_plane(ax, coefs, xlim=(-2.0, 2.0), ylim=(-2.0, 2.0), zlim=(-2.0, 2.0), color="orange"):
    a, b, c, d = coefs
    if np.abs(a) > np.abs(b) and np.abs(a) > np.abs(c):
        (y, z) = np.meshgrid(np.linspace(*ylim, 21), np.linspace(*zlim, 21))
        x = (d/a) - (b/a)*y - (c/a)*z
        x[(x > xlim[1]) | (x < xlim[0])] = np.nan
    elif np.abs(b) > np.abs(a) and np.abs(b) > np.abs(c):
        (x, z) = np.meshgrid(np.linspace(*xlim, 21), np.linspace(*zlim, 21))
        y = (d/b) - (a/b)*x - (c/b)*z
        y[(y > ylim[1]) | (y < ylim[0])] = np.nan
    else:
        (x, y) = np.meshgrid(np.linspace(*xlim, 21), np.linspace(*ylim, 21))
        z = (d/c) - (a/c)*x - (b/c)*y
        z[(z > zlim[1]) | (z < zlim[0])] = np.nan
    ax.plot_surface(x, y, z, color=color, alpha=0.3)

In [ ]:
def init_fig(xlim=(-2.0, 2.0), ylim=(-2.0, 2.0), zlim=(-2.0, 2.0), figsize=(5.0, 5.0), textwidth="2.0in"):
    plt.close("all")
    with plt.ioff():
        fig = plt.figure(figsize=figsize)
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    ax = fig.add_subplot(111, projection="3d")
    ax.set_clip_on(False)
    ax.set_proj_type("persp", focal_length=0.33)
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.set_zlim(*zlim)
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")
    ax.set_zlabel("$z$")
    ax.set_aspect("equal")
    ax.grid(False)
    text = Output(layout=Layout(width=textwidth))
    return fig, ax, text

def draw_axes(ax, xlim=(-2.0, 2.0), ylim=(-2.0, 2.0), zlim=(-2.0, 2.0)):
    ax.scatter(0.0, 0.0, 0.0, color="black", s=5.0)
    ax.plot([0.0, xlim[1]], [0.0, 0.0], [0.0, 0.0], color="darkred", linewidth=0.25)
    ax.plot([0.0, 0.0], [0.0, ylim[1]], [0.0, 0.0], color="darkgreen", linewidth=0.25)
    ax.plot([0.0, 0.0], [0.0, 0.0], [0.0, zlim[1]], color="darkblue", linewidth=0.25)

def draw_vec(ax, text, vect, name, color="black", s=0.03, o=vec(0.0, 0.0, 0.0), xlim=(-2.0, 2.0), ylim=(-2.0, 2.0), zlim=(-2.0, 2.0)):
    ax.quiver(*o, *vect, color=color, arrow_length_ratio=vec_ratio_3d(vect, s, xlim, ylim, zlim))
    ax.text(*vec_scale_3d(vect, o, s, xlim, ylim, zlim), f"$\\mathbf{{{name}}}$", color=color, va="center", ha="center")
    with text:
        display(Markdown(f"$ {{ \\color{{{color}}} \\mathbf{{{name}}} }} = {{ \\color{{{color}}} " + vec_to_str(vect) + " } $"))

def draw_span_dim1(ax, vect, color, xlim=(-2.0, 2.0), ylim=(-2.0, 2.0), zlim=(-2.0, 2.0)):
    a, b, c = vect
    if np.abs(a) > np.abs(b) and np.abs(a) > np.abs(c):
        ax.plot([xlim[0], xlim[1]], [xlim[0]*b/a, xlim[1]*b/a], [xlim[0]*c/a, xlim[1]*c/a], color=color, linewidth=2.0, alpha=0.5)
    elif np.abs(b) > np.abs(a) and np.abs(b) > np.abs(c):
        ax.plot([ylim[0]*a/b, ylim[1]*a/b], [ylim[0], ylim[1]], [ylim[0]*c/b, ylim[1]*c/b], color=color, linewidth=2.0, alpha=0.5)
    else:
        ax.plot([zlim[0]*a/c, zlim[1]*a/c], [zlim[0]*b/c, zlim[1]*b/c], [zlim[0], zlim[1]], color=color, linewidth=2.0, alpha=0.5)

def draw_span_dim2(ax, vect1, vect2, color, xlim=(-2.0, 2.0), ylim=(-2.0, 2.0), zlim=(-2.0, 2.0)):
    draw_plane(ax, (*np.cross(vect1, vect2), 0.0), xlim=xlim, ylim=ylim, zlim=zlim, color=color)

def disp_fig(fig, text):
    fig.tight_layout()
    display(widgets.HBox([widgets.VBox([widgets.Box(layout=Layout(flex="1 1 auto", height="auto")), text, widgets.Box(layout=Layout(flex="1 1 0%", height="auto"))]), fig.canvas]))

In [ ]:
xlim, ylim, zlim = (-3.0, 3.0), (-3.0, 3.0), (-3.0, 3.0)

u = vec(1, 0, -1)
v = vec(3, 0, 1)
w = vec(1, 1, 1)
r = vec(2, 2, 0)

fig, ax, text = init_fig(xlim=xlim, ylim=ylim, zlim=zlim)

draw_axes(ax, xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, u, "u", "blue", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, v, "v", "green", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, w, "w", "red", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, r, "r", "magenta", xlim=xlim, ylim=ylim, zlim=zlim)

draw_span_dim1(ax, u, "orange", xlim=xlim, ylim=ylim, zlim=zlim)
draw_span_dim2(ax, u, v, "orange", xlim=xlim, ylim=ylim, zlim=zlim)

disp_fig(fig, text)

In [ ]:
xlim, ylim, zlim = (-4.0, 4.0), (-4.0, 4.0), (-4.0, 4.0)

u = vec(1, 2, 3)
v = vec(3, 0, 1)
w = vec(4, 2, 4)

fig, ax, text = init_fig(xlim=xlim, ylim=ylim, zlim=zlim)

draw_axes(ax, xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, u, "u", "blue", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, v, "v", "green", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, w, "w", "red", xlim=xlim, ylim=ylim, zlim=zlim)

draw_span_dim1(ax, u, "orange", xlim=xlim, ylim=ylim, zlim=zlim)
draw_span_dim2(ax, u, v, "orange", xlim=xlim, ylim=ylim, zlim=zlim)

disp_fig(fig, text)

In [ ]:
xlim, ylim, zlim = (-4.0, 4.0), (-4.0, 4.0), (-4.0, 4.0)

u = vec(1, 0, 0)
v = vec(3, 0, 1)
w = vec(4, 0, 1)
r = vec(0, 0, 1)
q = vec(0, 0, -3)

fig, ax, text = init_fig(xlim=xlim, ylim=ylim, zlim=zlim)

draw_axes(ax, xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, u, "u", "blue", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, v, "v", "green", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, w, "w", "red", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, r, "r", "magenta", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, q, "q", "brown", xlim=xlim, ylim=ylim, zlim=zlim)

draw_span_dim1(ax, u, "orange", xlim=xlim, ylim=ylim, zlim=zlim)
draw_span_dim2(ax, u, v, "orange", xlim=xlim, ylim=ylim, zlim=zlim)

disp_fig(fig, text)

In [ ]:
xlim, ylim, zlim = (-3.0, 3.0), (-3.0, 3.0), (-3.0, 3.0)

u = vec(-1, -1, 1)
v = vec(0, 2, 0)
w = vec(1, 3, -1)
r = vec(-1, 1, 1)

fig, ax, text = init_fig(xlim=xlim, ylim=ylim, zlim=zlim)

draw_axes(ax, xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, u, "u", "blue", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, v, "v", "green", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, w, "w", "red", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, r, "r", "magenta", xlim=xlim, ylim=ylim, zlim=zlim)

draw_span_dim1(ax, u, "orange", xlim=xlim, ylim=ylim, zlim=zlim)
draw_span_dim2(ax, u, v, "orange", xlim=xlim, ylim=ylim, zlim=zlim)

disp_fig(fig, text)

In [ ]:
xlim, ylim, zlim = (-5.0, 5.0), (-5.0, 5.0), (-5.0, 5.0)

u = vec(0, -5, 5)
v = vec(0, 3, -3)
w = vec(0, 2, -2)
r = vec(0, -1, 1)

fig, ax, text = init_fig(xlim=xlim, ylim=ylim, zlim=zlim)

draw_axes(ax, xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, u, "u", "blue", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, v, "v", "green", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, w, "w", "red", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, r, "r", "magenta", xlim=xlim, ylim=ylim, zlim=zlim)

draw_span_dim1(ax, u, "orange", xlim=xlim, ylim=ylim, zlim=zlim)

disp_fig(fig, text)

In [ ]:
xlim, ylim, zlim = (-3.0, 3.0), (-3.0, 3.0), (-3.0, 3.0)

u = vec(1, 0, 0)
v = vec(3, 1, 1)
w = vec(0, 0, 1)

fig, ax, text = init_fig(xlim=xlim, ylim=ylim, zlim=zlim)

draw_axes(ax, xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, u, "u", "blue", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, v, "v", "green", xlim=xlim, ylim=ylim, zlim=zlim)
draw_vec(ax, text, w, "w", "red", xlim=xlim, ylim=ylim, zlim=zlim)

draw_span_dim1(ax, u, "orange", xlim=xlim, ylim=ylim, zlim=zlim)
draw_span_dim2(ax, u, v, "orange", xlim=xlim, ylim=ylim, zlim=zlim)

disp_fig(fig, text)